## Dataset Loading

In [103]:
# Loading raw data
import pandas as pd
fraud_dataset_path = r"..\src\data\raw\Fraud_Data.csv"
# device_duplicate_path = r"..\src\data\cleaned\intermediate\duplicate_device_usage.csv"
ip_address_to_country = r"..\src\data\raw\IpAddress_to_Country.csv"

# ip_address_to_country = r"..\src\data\raw\IpAddress_to_Country.csv"
# fraud_dataset_path = "/content/drive/MyDrive/10Academy/Week8and9/Fraud_Data.csv"

df_fraud = pd.read_csv(fraud_dataset_path)
# df_device_duplicate = pd.read_csv(device_duplicate_path)
df_ip = pd.read_csv(ip_address_to_country)
# df_ip = pd.read_csv(ip_address_to_country)



## Dataset Merging

In [104]:
# Merging Fraud Dataset and Ip table
def ip_to_country_converter(row):
  ip = row['ip_address']
  country = df_ip[(df_ip['lower_bound_ip_address'] <=ip) & (df_ip['upper_bound_ip_address'] >= ip)]['country'].values
  if len(country) > 0:
    return country[0]

df_fraud['country'] = df_fraud.apply(ip_to_country_converter, axis=1)

In [105]:
df_fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,None
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States


## Data type conversion

In [106]:
# Convert signup and purchase time to datetime
def convert_to_datetime(df):
    # Convert 'at' column to datetime object
    df['purchase_time_dt'] = pd.to_datetime(df['purchase_time'], errors='coerce')
    df['signup_time_dt'] = pd.to_datetime(df['signup_time'], errors='coerce')


convert_to_datetime(df_fraud)


In [107]:
df_fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,purchase_time_dt,signup_time_dt
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,2015-04-18 02:47:11,2015-02-24 22:55:49
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,2015-06-08 01:38:54,2015-06-07 20:39:50
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,2015-01-01 18:52:45,2015-01-01 18:52:44
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,None,2015-05-04 13:54:50,2015-04-28 21:13:25
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States,2015-09-09 18:40:53,2015-07-21 07:09:52


## Feature Engineering

### Device Frequency

In [108]:
df_device_duplicate = pd.DataFrame(df_fraud.groupby(by="device_id").device_id.count())
df_device_duplicate.rename(columns={'device_id':'freq_count'}, inplace=True)
df_device_duplicate.reset_index(level=0, inplace= True)

In [109]:
df_device_duplicate.head()

,device_id,freq_count
0,AAALBGNHHVMKG,1
1,AAAWIHVCQELTP,1
2,AAAXJHWCLISKY,1
3,AAAXXOZJRZRAO,11
4,AABFGRPBQHWFQ,1


In [110]:
df_fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,purchase_time_dt,signup_time_dt
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,2015-04-18 02:47:11,2015-02-24 22:55:49
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,2015-06-08 01:38:54,2015-06-07 20:39:50
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,2015-01-01 18:52:45,2015-01-01 18:52:44
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,None,2015-05-04 13:54:50,2015-04-28 21:13:25
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States,2015-09-09 18:40:53,2015-07-21 07:09:52


In [111]:
# Merging device duplicate dataframe with fraud dataframe

df_fraud = df_fraud.merge(df_device_duplicate, on= "device_id")

In [112]:
df_fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,purchase_time_dt,signup_time_dt,freq_count
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,2015-04-18 02:47:11,2015-02-24 22:55:49,1
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,2015-06-08 01:38:54,2015-06-07 20:39:50,1
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,2015-01-01 18:52:45,2015-01-01 18:52:44,12
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,None,2015-05-04 13:54:50,2015-04-28 21:13:25,1
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States,2015-09-09 18:40:53,2015-07-21 07:09:52,1


The cell below changes freq_count into bins

In [113]:
df_fraud["freq_count_bins"] = df_fraud.freq_count.apply(lambda x :
                                                str(x) if x <5 else
                                                "5-10" if x>=5 and x<=10 else
                                                "11-15" if x>10 and x<=15 else
                                                "> 15")

In [114]:
df_fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,purchase_time_dt,signup_time_dt,freq_count,freq_count_bins
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,2015-04-18 02:47:11,2015-02-24 22:55:49,1,1
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,2015-06-08 01:38:54,2015-06-07 20:39:50,1,1
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,2015-01-01 18:52:45,2015-01-01 18:52:44,12,11-15
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,None,2015-05-04 13:54:50,2015-04-28 21:13:25,1,1
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States,2015-09-09 18:40:53,2015-07-21 07:09:52,1,1


In [115]:
df_fraud['freq_count_bins'].unique()

array(['1', '11-15', '5-10', '2', '> 15', '4', '3'], dtype=object)

### Extracting Date time features

In [116]:
# Extract month, week, hour of the day
import calendar

def extract_time_features(df):
    # Extract month
    df['purchase_month'] = df.purchase_time_dt.apply(lambda x: calendar.month_name[x.month])
    df['purchase_week'] = df.purchase_time_dt.apply(lambda x: calendar.day_name[x.weekday()])
    df['purchase_hour'] = df.purchase_time_dt.apply(lambda x: x.hour)
    df['seconds_since_signup'] = (df.purchase_time_dt - df.signup_time_dt).apply(lambda x: x.total_seconds())

extract_time_features(df_fraud)

In [117]:
df_fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,purchase_time_dt,signup_time_dt,freq_count,freq_count_bins,purchase_month,purchase_week,purchase_hour,seconds_since_signup
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,Japan,2015-04-18 02:47:11,2015-02-24 22:55:49,1,1,April,Saturday,2,4506682.0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,United States,2015-06-08 01:38:54,2015-06-07 20:39:50,1,1,June,Monday,1,17944.0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,United States,2015-01-01 18:52:45,2015-01-01 18:52:44,12,11-15,January,Thursday,18,1.0
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,None,2015-05-04 13:54:50,2015-04-28 21:13:25,1,1,May,Monday,13,492085.0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,United States,2015-09-09 18:40:53,2015-07-21 07:09:52,1,1,September,Wednesday,18,4361461.0


### Country Risk

In [118]:
# Getting country risk
country_risk_df = pd.DataFrame(df_fraud.groupby(by='country')['class'].mean().sort_values(ascending=False))
country_risk_df = country_risk_df.reset_index().rename(columns={'class': 'country_risk_values'})

In [119]:
# Calculate country risk bins
country_risk_df['country_risk_bins'] = country_risk_df['country_risk_values'].apply(lambda x:
                                                            "Very high risk" if x > 0.25 else
                                                            "High risk" if x > 0.05 else
                                                            "Medium risk" if x > 0.01 else
                                                            "Low risk")

In [120]:
country_risk_df.head()

,country,country_risk_values,country_risk_bins
0,Turkmenistan,1.000000,Very high risk
1,Namibia,0.434783,Very high risk
2,Sri Lanka,0.419355,Very high risk
3,Luxembourg,0.388889,Very high risk
4,Virgin Islands (U.S.),0.333333,Very high risk


In [121]:
# Merge df_fraud and country_risk_df
df_fraud = df_fraud.merge(country_risk_df, on='country')

In [122]:
df_fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,...,purchase_time_dt,signup_time_dt,freq_count,freq_count_bins,purchase_month,purchase_week,purchase_hour,seconds_since_signup,country_risk_values,country_risk_bins
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,...,2015-04-18 02:47:11,2015-02-24 22:55:49,1,1,April,Saturday,2,4506682.0,0.097865,High risk
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,...,2015-06-08 01:38:54,2015-06-07 20:39:50,1,1,June,Monday,1,17944.0,0.095626,High risk
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,...,2015-01-01 18:52:45,2015-01-01 18:52:44,12,11-15,January,Thursday,18,1.0,0.095626,High risk
3,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,...,2015-09-09 18:40:53,2015-07-21 07:09:52,1,1,September,Wednesday,18,4361461.0,0.095626,High risk
4,159135,2015-05-21 06:03:03,2015-07-09 08:05:14,42,ALEYXFXINSXLZ,Ads,Chrome,M,18,2.809315e+09,...,2015-07-09 08:05:14,2015-05-21 06:03:03,1,1,July,Thursday,8,4240931.0,0.116975,High risk


In [ ]:
# # Calculate country risk bins
# country_risk_df['country_risk_bins'] = country_risk_df['country_risk_values'].apply(lambda x:
#                                                             "Very high risk" if x > 0.25 else
#                                                             "High risk" if x > 0.05 else
#                                                             "Medium risk" if x > 0.01 else
#                                                             "Low risk")


In [ ]:
# df_fraud = df_fraud.merge(country_risk_df, on="country")

### Age Bins

In [123]:
df_fraud["age_bins"] =  df_fraud["age"].apply(lambda x:
                                              "< 40" if x < 40 else
                                              "40 - 49" if x<50 else
                                              "50 - 59" if x<60 else
                                              "60 - 69" if x < 70 else
                                              "> 70")

In [124]:
df_fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,...,signup_time_dt,freq_count,freq_count_bins,purchase_month,purchase_week,purchase_hour,seconds_since_signup,country_risk_values,country_risk_bins,age_bins
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,...,2015-02-24 22:55:49,1,1,April,Saturday,2,4506682.0,0.097865,High risk,< 40
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,...,2015-06-07 20:39:50,1,1,June,Monday,1,17944.0,0.095626,High risk,50 - 59
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,...,2015-01-01 18:52:44,12,11-15,January,Thursday,18,1.0,0.095626,High risk,50 - 59
3,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,...,2015-07-21 07:09:52,1,1,September,Wednesday,18,4361461.0,0.095626,High risk,40 - 49
4,159135,2015-05-21 06:03:03,2015-07-09 08:05:14,42,ALEYXFXINSXLZ,Ads,Chrome,M,18,2.809315e+09,...,2015-05-21 06:03:03,1,1,July,Thursday,8,4240931.0,0.116975,High risk,< 40


## Save the cleaned and processed dataset

In [ ]:
df_fraud.to_csv(r"../src/data/cleaned/processed/ecommerce_fraud_dataset.csv", index=False)